In [1]:
import pandas as pd
import numpy as np
import requests
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import statsmodels.api as sm
from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from tpot import TPOTClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import LabelEncoder
from tpot import TPOTClassifier
from sklearn.tree import DecisionTreeClassifier
from mord import OrdinalRidge
import xgboost as xgb
from keras.layers import Input
from keras_tuner import RandomSearch
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score
import shap
from tabulate import tabulate
import requests
from io import StringIO

In [2]:
# The URL of the raw dataset on GitHub
url = "https://raw.githubusercontent.com/abactat/BC-Project/main/data/processed/train_data.csv?token=GHSAT0AAAAAACC4ZCNLWABIZIRO3TNG2HUSZGNLOWQ"

# Send an HTTP GET request to fetch the content of the raw dataset
response = requests.get(url)

# Check if the request was successful (status code 200 means success)
if response.status_code == 200:
    # Read the content as a pandas DataFrame and assign it to the 'train' variable
    train = pd.read_csv(StringIO(response.text))
    print("Dataset downloaded and loaded into 'train' successfully.")
else:
    print(f"Failed to download the dataset. Status code: {response.status_code}")
    
# The URL of the raw dataset on GitHub
url = "https://raw.githubusercontent.com/abactat/BC-Project/main/data/processed/val_data.csv?token=GHSAT0AAAAAACC4ZCNLQ25HM5AEQIEEW4XSZGNLPDA"

# Send an HTTP GET request to fetch the content of the raw dataset
response = requests.get(url)

# Check if the request was successful (status code 200 means success)
if response.status_code == 200:
    # Read the content as a pandas DataFrame and assign it to the 'valid' variable
    valid = pd.read_csv(StringIO(response.text))
    print("Dataset downloaded and loaded into 'valid' successfully.")
else:
    print(f"Failed to download the dataset. Status code: {response.status_code}")
    
# The URL of the raw dataset on GitHub
url = "https://raw.githubusercontent.com/abactat/BC-Project/main/data/processed/test_data.csv?token=GHSAT0AAAAAACC4ZCNKIY5A23DKULFKCT56ZGNLPOA"

# Send an HTTP GET request to fetch the content of the raw dataset
response = requests.get(url)

# Check if the request was successful (status code 200 means success)
if response.status_code == 200:
    # Read the content as a pandas DataFrame and assign it to the 'train' variable
    test = pd.read_csv(StringIO(response.text))
    print("Dataset downloaded and loaded into 'test' successfully.")
else:
    print(f"Failed to download the dataset. Status code: {response.status_code}")

Dataset downloaded and loaded into 'train' successfully.
Dataset downloaded and loaded into 'valid' successfully.
Dataset downloaded and loaded into 'test' successfully.


In [3]:
# Convert variables to numeric in the train dataset
variables_to_convert = train.columns.drop('Date')
train[variables_to_convert] = train[variables_to_convert].apply(pd.to_numeric, errors='coerce')

# Convert variables to numeric in the valid dataset
valid[variables_to_convert] = valid[variables_to_convert].apply(pd.to_numeric, errors='coerce')

In [4]:
train['Direction'] = np.select([
    train['Difference'] > 0,
    train['Difference'] < 0,
    train['Difference'] == 0
], [
    1,  # Increase
    -1,  # Decrease
    0   # Hold
], default=-1)

valid['Direction'] = np.select([
    valid['Difference'] > 0,
    valid['Difference'] < 0,
    valid['Difference'] == 0
], [
    1,  # Increase
    -1,  # Decrease
    0   # Hold
], default=-1)

In [5]:
print(train.columns)

Index(['Increase', 'Decrease', 'Level', 'CPI', 'Unemployment Rate',
       'Retail Sales', 'Durable Goods Orders', 'Short-Term Treasury Bond Rate',
       'Long-Term Treasury Bond Rate', 'Bank Reserves', 'LEI', 'CEI', 'LAG',
       'Treasury Deposits', 'Average Hourly Earnings', 'Net Sentiment Score',
       'Positive Frequency', 'Negative Frequency', 'Word Count',
       'Standardized Sentiment Score', 'Short-Term Treasury Diff',
       'LEI_RollingMean', 'CEI_RollingMean', 'LAG_RollingMean',
       'Retail Sales_RollingMean', 'Durable Goods Orders_RollingMean',
       'almost', 'anticipated', 'apparent', 'apparently', 'appeared',
       'appearing', 'assumed', 'assumption', 'attain', 'attained',
       'attractive', 'believed', 'bolstered', 'boost', 'cautious',
       'cautiously', 'could', 'depend', 'desired', 'easier', 'encouragement',
       'favorable', 'good', 'honor', 'improved', 'improvement', 'leading',
       'might', 'nearly', 'opportunity', 'optimistic', 'perhaps', 'positi

###Directional Prediction

###Ordinal Ridge Regression

In [6]:
# Prepare the data for the model
X_train = train.drop(columns=['Difference', 'Increase', 'Decrease', 'Direction', 'Date'])
y_train = train['Direction']  # Use the 'Direction' variable as the target

X_valid = valid.drop(columns=['Difference', 'Increase', 'Decrease', 'Direction', 'Date'])
y_valid = valid['Direction']  # Use the 'Direction' variable as the target

# Center and scale the independent variables
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

# Initialize and train the ordinal logistic regression model
ordinal_model = OrdinalRidge()
ordinal_model.fit(X_train_scaled, y_train)

# Make predictions on the validation set
y_valid_pred = ordinal_model.predict(X_valid_scaled)

# Make predictions on the training set
y_train_pred = ordinal_model.predict(X_train_scaled)

In [7]:
# Calculate Mean Squared Error (MSE) for training set
mse_train = mean_squared_error(y_train, y_train_pred)
print("Training Mean Squared Error (MSE):", mse_train)

# Calculate Mean Squared Error (MSE) for validation set
mse_valid = mean_squared_error(y_valid, y_valid_pred)
print("Validation Mean Squared Error (MSE):", mse_valid)

# Calculate Root Mean Squared Error (RMSE) for training set
rmse_train = np.sqrt(mse_train)
print("Training Root Mean Squared Error (RMSE):", rmse_train)

# Calculate Root Mean Squared Error (RMSE) for validation set
rmse_valid = np.sqrt(mse_valid)
print("Validation Root Mean Squared Error (RMSE):", rmse_valid)

# Calculate R-squared (R^2) for training set
r2_train = r2_score(y_train, y_train_pred)
print("Training R^2:", r2_train)

# Calculate R-squared (R^2) for validation set
r2_valid = r2_score(y_valid, y_valid_pred)
print("Validation R^2:", r2_valid)

# Calculate adjusted R-squared for training set
n_train = X_train.shape[0]  # Number of samples in training set
p_train = X_train.shape[1]  # Number of features in training set
adj_r2_train = 1 - ((1 - r2_train) * (n_train - 1) / (n_train - p_train - 1))
print("Training Adjusted R^2:", adj_r2_train)

# Calculate adjusted R-squared for validation set
n_valid = X_valid.shape[0]  # Number of samples in validation set
p_valid = X_valid.shape[1]  # Number of features in validation set
adj_r2_valid = 1 - ((1 - r2_valid) * (n_valid - 1) / (n_valid - p_valid - 1))
print("Validation Adjusted R^2:", adj_r2_valid)

Training Mean Squared Error (MSE): 0.16666666666666666
Validation Mean Squared Error (MSE): 0.375
Training Root Mean Squared Error (RMSE): 0.408248290463863
Validation Root Mean Squared Error (RMSE): 0.6123724356957945
Training R^2: 0.4344103838718586
Validation R^2: 0.08474576271186429
Training Adjusted R^2: -0.11368676990180426
Validation Adjusted R^2: 1.296490809262354


In [8]:
# Initialize and train the ordinal logistic regression model
ordinal_model = OrdinalRidge()
ordinal_model.fit(X_train_scaled, y_train)

# Make predictions on the validation set
y_valid_pred = ordinal_model.predict(X_valid_scaled)

# Make predictions on the training set
y_train_pred = ordinal_model.predict(X_train_scaled)

# Convert the predicted values to ordinal categories
y_train_pred_category = np.round(y_train_pred).astype(int)
y_valid_pred_category = np.round(y_valid_pred).astype(int)

# Calculate accuracy for training and validation sets
accuracy_train = accuracy_score(y_train, y_train_pred_category)
accuracy_valid = accuracy_score(y_valid, y_valid_pred_category)

print("Training Accuracy:", accuracy_train)
print("Validation Accuracy:", accuracy_valid)

Training Accuracy: 0.8333333333333334
Validation Accuracy: 0.625


In [9]:
# Access the coefficients and intercepts
coefficients = ordinal_model.coef_
intercepts = ordinal_model.intercept_

# Display coefficients and intercepts
print("Coefficients:")
print(coefficients)

print("Intercepts:")
print(intercepts)

Coefficients:
[ 3.60006388e-03 -1.71299359e-01 -1.12706905e-01  2.01311442e-01
  6.74227573e-02 -2.09685714e-02 -1.21236502e-01 -9.86106910e-02
 -1.04473429e-01 -4.66635565e-02 -5.41486227e-03  9.44796914e-03
 -2.11858784e-01  5.39941527e-02  8.54444656e-04 -1.67977011e-02
 -7.33325297e-02  5.39941527e-02  3.10616053e-01  2.22308439e-02
  3.82292476e-02  2.37956393e-02  3.44796934e-01 -3.08829946e-01
 -3.05042422e-02 -5.88170702e-02  9.40186556e-02 -6.06642072e-02
 -3.85541953e-02 -9.79729188e-02  3.41971237e-02 -8.13291259e-02
  1.81353251e-02  2.84454140e-04  4.71949831e-02 -8.73722857e-02
 -1.44271280e-01 -1.04817260e-02 -7.92461716e-02 -3.99575900e-02
  3.29054361e-01 -1.60710650e-01 -3.79804938e-02  1.46356590e-01
 -7.84034746e-02  3.75905859e-02  1.64881896e-01  3.48058658e-02
 -1.29782125e-01 -5.81597964e-02 -2.52587886e-02 -2.63945811e-01
  4.15903549e-02 -7.78876402e-02  4.74711017e-02  1.48991360e-01
  5.51856448e-02 -7.33008302e-02  1.37824139e-02  9.43627895e-02
 -5.8235929

###XGBoost Model

In [10]:
# Prepare the data for the model
X_train = train.drop(columns=['Difference', 'Increase', 'Decrease', 'Direction', 'Date'])
y_train = train['Direction']  # Use the 'Direction' variable as the target

X_valid = valid.drop(columns=['Difference', 'Increase', 'Decrease', 'Direction', 'Date'])
y_valid = valid['Direction']  # Use the 'Direction' variable as the target

# Initialize and train the XGBoost model
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train, y_train)

# Make predictions on the training set
y_train_pred = xgb_model.predict(X_train)

# Make predictions on the validation set
y_valid_pred = xgb_model.predict(X_valid)

In [11]:
# Calculate Mean Squared Error (MSE) for training set
mse_train = mean_squared_error(y_train, y_train_pred)
print("Training Mean Squared Error (MSE):", mse_train)

# Calculate Mean Squared Error (MSE) for validation set
mse_valid = mean_squared_error(y_valid, y_valid_pred)
print("Validation Mean Squared Error (MSE):", mse_valid)

# Calculate Root Mean Squared Error (RMSE) for training set
rmse_train = np.sqrt(mse_train)
print("Training Root Mean Squared Error (RMSE):", rmse_train)

# Calculate Root Mean Squared Error (RMSE) for validation set
rmse_valid = np.sqrt(mse_valid)
print("Validation Root Mean Squared Error (RMSE):", rmse_valid)

# Calculate R-squared (R^2) for training set
r2_train = r2_score(y_train, y_train_pred)
print("Training R^2:", r2_train)

# Calculate R-squared (R^2) for validation set
r2_valid = r2_score(y_valid, y_valid_pred)
print("Validation R^2:", r2_valid)

# Calculate adjusted R-squared for training set
n_train = X_train.shape[0]  # Number of samples in training set
p_train = X_train.shape[1]  # Number of features in training set
adj_r2_train = 1 - ((1 - r2_train) * (n_train - 1) / (n_train - p_train - 1))
print("Training Adjusted R^2:", adj_r2_train)

# Calculate adjusted R-squared for validation set
n_valid = X_valid.shape[0]  # Number of samples in validation set
p_valid = X_valid.shape[1]  # Number of features in validation set
adj_r2_valid = 1 - ((1 - r2_valid) * (n_valid - 1) / (n_valid - p_valid - 1))
print("Validation Adjusted R^2:", adj_r2_valid)

Training Mean Squared Error (MSE): 2.487542025649244e-07
Validation Mean Squared Error (MSE): 0.3220936950197386
Training Root Mean Squared Error (RMSE): 0.000498752646674606
Validation Root Mean Squared Error (RMSE): 0.5675329902479138
Training R^2: 0.9999991558432364
Validation R^2: 0.21387301554504468
Training Adjusted R^2: 0.9999983377944139
Validation Adjusted R^2: 1.2546608541192108


In [12]:
# Initialize and train the XGBoost model
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train, y_train)

# Extract feature importance
feature_importance = xgb_model.feature_importances_

# Create a dictionary to map feature names to their importance scores
feature_importance_dict = {feature_name: importance_score for feature_name, importance_score in zip(X_train.columns, feature_importance)}

# Sort the feature importance dictionary in descending order based on importance scores
sorted_feature_importance = dict(sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True))

# Print the feature importance scores in descending order
print("Feature Importance:")
for feature_name, importance_score in sorted_feature_importance.items():
    print(f"{feature_name}: {importance_score:.4f}")

Feature Importance:
Short-Term Treasury Diff: 0.3441
Short-Term Treasury Bond Rate: 0.1476
LAG_RollingMean: 0.0661
perhaps: 0.0539
anticipated: 0.0461
Unemployment Rate: 0.0452
suggested: 0.0291
boost: 0.0276
Bank Reserves: 0.0245
Retail Sales: 0.0228
LEI: 0.0225
CEI: 0.0225
progress: 0.0216
Treasury Deposits: 0.0206
Net Sentiment Score: 0.0135
Level: 0.0124
Word Count: 0.0096
encouragement: 0.0095
appeared: 0.0084
Long-Term Treasury Bond Rate: 0.0081
CPI: 0.0078
cautious: 0.0078
LEI_RollingMean: 0.0047
Positive Frequency: 0.0047
improved: 0.0046
Durable Goods Orders_RollingMean: 0.0032
unexpected: 0.0027
improvement: 0.0026
could: 0.0018
positive: 0.0009
stabilization: 0.0005
LAG: 0.0005
unclear: 0.0004
uncertainty: 0.0003
attractive: 0.0003
risk: 0.0002
possibility: 0.0002
possible: 0.0002
Negative Frequency: 0.0001
favorable: 0.0001
rebound: 0.0001
CEI_RollingMean: 0.0001
good: 0.0001
satisfactory: 0.0001
Average Hourly Earnings: 0.0001
opportunity: 0.0000
bolstered: 0.0000
apparent

In [13]:
# Initialize and train the XGBoost model
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train, y_train)

# Make predictions on the training set
y_train_pred = xgb_model.predict(X_train)

# Make predictions on the validation set
y_valid_pred = xgb_model.predict(X_valid)

# Convert the predicted values to ordinal categories
y_train_pred_category = np.round(y_train_pred).astype(int)
y_valid_pred_category = np.round(y_valid_pred).astype(int)

# Calculate accuracy for training and validation sets
accuracy_train = accuracy_score(y_train, y_train_pred_category)
accuracy_valid = accuracy_score(y_valid, y_valid_pred_category)

print("Training Accuracy:", accuracy_train)
print("Validation Accuracy:", accuracy_valid)

Training Accuracy: 1.0
Validation Accuracy: 0.5833333333333334


###RNN Model

In [14]:
# Preprocess the data
X_train = train.drop(columns=['Direction', 'Increase', 'Decrease', 'Difference', 'Date'])
y_train = train['Direction']

X_valid = valid.drop(columns=['Direction', 'Increase', 'Decrease', 'Difference', 'Date'])
y_valid = valid['Direction']

# Encode the ordinal 'Direction' variable to numerical values (e.g., -1, 0, 1)
y_train_encoded = pd.Series(np.where(y_train == 'Decrease', -1, np.where(y_train == 'Hold', 0, 1)))
y_valid_encoded = pd.Series(np.where(y_valid == 'Decrease', -1, np.where(y_valid == 'Hold', 0, 1)))

# Center and scale the independent variables
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

# Build the RNN model with additional modifications to obtain feature importances
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('lstm_units', min_value=32, max_value=128, step=32),
                   input_shape=(sequence_length, 1),
                   return_sequences=True))
    model.add(LSTM(units=hp.Int('lstm_units', min_value=16, max_value=64, step=16)))
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4])),
                  metrics=['accuracy'])

    return model

# Prepare the input sequences for the RNN model (assuming sequence_length is defined)
def prepare_sequences(data, sequence_length):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length + 1):
        sequences.append(data[i:i+sequence_length])
        targets.append(data[i+sequence_length-1])
    return np.array(sequences), np.array(targets)

sequence_length = 10  # Define the length of the input sequence
X_train_seq, y_train_seq = prepare_sequences(y_train_encoded, sequence_length)
X_valid_seq, y_valid_seq = prepare_sequences(y_valid_encoded, sequence_length)

# Build the RNN model with additional modifications to obtain feature importances
model = Sequential()
model.add(LSTM(64, input_shape=(sequence_length, 1), return_sequences=True))
model.add(LSTM(32))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_seq, y_train_seq, epochs=10, batch_size=32, validation_data=(X_valid_seq, y_valid_seq))

# Evaluate the model on the validation set
y_train_pred = model.predict(X_train_seq).argmax(axis=1)
y_valid_pred = model.predict(X_valid_seq).argmax(axis=1)

# Calculate Mean Squared Error (MSE) for training and validation sets
mse_train = mean_squared_error(y_train_seq, y_train_pred)
mse_valid = mean_squared_error(y_valid_seq, y_valid_pred)
print("Training Mean Squared Error (MSE):", mse_train)
print("Validation Mean Squared Error (MSE):", mse_valid)

# Calculate Root Mean Squared Error (RMSE) for training and validation sets
rmse_train = np.sqrt(mse_train)
rmse_valid = np.sqrt(mse_valid)
print("Training Root Mean Squared Error (RMSE):", rmse_train)
print("Validation Root Mean Squared Error (RMSE):", rmse_valid)

# Calculate R-squared (R^2) for training and validation sets
r2_train = r2_score(y_train_seq, y_train_pred)
r2_valid = r2_score(y_valid_seq, y_valid_pred)
print("Training R^2:", r2_train)
print("Validation R^2:", r2_valid)

# Calculate the number of samples and features
n_train, p_train = X_train_seq.shape[0], X_train_seq.shape[1]
n_valid, p_valid = X_valid_seq.shape[0], X_valid_seq.shape[1]

# Calculate adjusted R-squared for training and validation sets
adj_r2_train = 1 - ((1 - r2_train) * (n_train - 1) / (n_train - p_train - 1))
adj_r2_valid = 1 - ((1 - r2_valid) * (n_valid - 1) / (n_valid - p_valid - 1))
print("Training Adjusted R^2:", adj_r2_train)
print("Validation Adjusted R^2:", adj_r2_valid)

# Show training and validation set accuracy
accuracy_train = (y_train_seq == y_train_pred).mean()
accuracy_valid = (y_valid_seq == y_valid_pred).mean()
print("Training Accuracy:", accuracy_train)
print("Validation Accuracy:", accuracy_valid)

Epoch 1/10
6/6 [==============================] - 5s 233ms/step - loss: 0.8856 - accuracy: 0.8251 - val_loss: 0.6356 - val_accuracy: 1.0000
Epoch 2/10
6/6 [==============================] - 0s 23ms/step - loss: 0.4718 - accuracy: 1.0000 - val_loss: 0.2386 - val_accuracy: 1.0000
Epoch 3/10
6/6 [==============================] - 0s 20ms/step - loss: 0.1387 - accuracy: 1.0000 - val_loss: 0.0406 - val_accuracy: 1.0000
Epoch 4/10
6/6 [==============================] - 0s 20ms/step - loss: 0.0217 - accuracy: 1.0000 - val_loss: 0.0067 - val_accuracy: 1.0000
Epoch 5/10
6/6 [==============================] - 0s 20ms/step - loss: 0.0042 - accuracy: 1.0000 - val_loss: 0.0021 - val_accuracy: 1.0000
Epoch 6/10
6/6 [==============================] - 0s 21ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 7/10
6/6 [==============================] - 0s 22ms/step - loss: 9.0957e-04 - accuracy: 1.0000 - val_loss: 7.2977e-04 - val_accuracy: 1.0000
Epoch 8/10
6/6 [==

In [15]:
# Build the RNN model with additional modifications to obtain feature importances
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('lstm_units', min_value=32, max_value=128, step=32),
                   input_shape=(sequence_length, 1),
                   return_sequences=True))
    model.add(LSTM(units=hp.Int('lstm_units', min_value=16, max_value=64, step=16)))
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4])),
                  metrics=['accuracy'])

    return model

# Define the hyperparameter search space
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Number of hyperparameter combinations to try
    executions_per_trial=1,
    directory='my_dir',  # Directory to store the results
    project_name='my_project'
)

# Search for the best hyperparameters
tuner.search(X_train_seq, y_train_seq, epochs=10, batch_size=32, validation_data=(X_valid_seq, y_valid_seq))

# Get the best model
best_model = tuner.get_best_models(1)[0]

# Print the summary of the best model
best_model.summary()

# Train the best model
best_model.fit(X_train_seq, y_train_seq, epochs=10, batch_size=32, validation_data=(X_valid_seq, y_valid_seq))

# Evaluate the best model on the validation set
y_train_pred = best_model.predict(X_train_seq).argmax(axis=1)
y_valid_pred = best_model.predict(X_valid_seq).argmax(axis=1)

# Calculate Mean Squared Error (MSE) for training and validation sets
mse_train = mean_squared_error(y_train_seq, y_train_pred)
mse_valid = mean_squared_error(y_valid_seq, y_valid_pred)
print("Training Mean Squared Error (MSE):", mse_train)
print("Validation Mean Squared Error (MSE):", mse_valid)

# Calculate Root Mean Squared Error (RMSE) for training and validation sets
rmse_train = np.sqrt(mse_train)
rmse_valid = np.sqrt(mse_valid)
print("Training Root Mean Squared Error (RMSE):", rmse_train)
print("Validation Root Mean Squared Error (RMSE):", rmse_valid)

# Calculate R-squared (R^2) for training and validation sets
r2_train = r2_score(y_train_seq, y_train_pred)
r2_valid = r2_score(y_valid_seq, y_valid_pred)
print("Training R^2:", r2_train)
print("Validation R^2:", r2_valid)

# Calculate the number of samples and features
n_train, p_train = X_train_seq.shape[0], X_train_seq.shape[1]
n_valid, p_valid = X_valid_seq.shape[0], X_valid_seq.shape[1]

# Calculate adjusted R-squared for training and validation sets
adj_r2_train = 1 - ((1 - r2_train) * (n_train - 1) / (n_train - p_train - 1))
adj_r2_valid = 1 - ((1 - r2_valid) * (n_valid - 1) / (n_valid - p_valid - 1))
print("Training Adjusted R^2:", adj_r2_train)
print("Validation Adjusted R^2:", adj_r2_valid)

# Show training and validation set accuracy
accuracy_train = (y_train_seq == y_train_pred).mean()
accuracy_valid = (y_valid_seq == y_valid_pred).mean()
print("Training Accuracy:", accuracy_train)
print("Validation Accuracy:", accuracy_valid)

# Get the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
print("Best Hyperparameters:")
print(best_hyperparameters.values)

INFO:tensorflow:Reloading Tuner from my_dir\my_project\tuner0.json
INFO:tensorflow:Oracle triggered exit
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 96)            37632     
                                                                 
 lstm_1 (LSTM)               (None, 96)                74112     
                                                                 
 dense (Dense)               (None, 3)                 291       
                                                                 
Total params: 112,035
Trainable params: 112,035
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
6/6 [==============================] - 4s 211ms/step - loss: 0.1457 - accuracy: 1.0000 - val_loss: 0.0056 - val_accuracy: 1.0000
Epoch 2/10
6/6 [==============================] - 0s 25ms/step - loss

In [16]:
# Build the RNN model with additional modifications to obtain feature importances
model = Sequential()
model.add(Input(shape=(sequence_length, 1)))  # Add an input layer with the desired input shape
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Show the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 10, 64)            16896     
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 29,411
Trainable params: 29,411
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Convert 'Date' column to datetime type
train['Date'] = pd.to_datetime(train['Date'])
valid['Date'] = pd.to_datetime(valid['Date'])

# Define a reference date (you can choose any date you prefer)
reference_date = pd.to_datetime('1990-01-01')

# Extract numerical representation of 'Date' column
train['Date'] = (train['Date'] - reference_date).dt.days
valid['Date'] = (valid['Date'] - reference_date).dt.days

# Preprocess the data
X_train = train.drop(columns=['Direction', 'Increase', 'Decrease', 'Difference'])
y_train = train['Direction']

X_valid = valid.drop(columns=['Direction', 'Increase', 'Decrease', 'Difference'])
y_valid = valid['Direction']

# Encode the ordinal 'Direction' variable to numerical values (e.g., -1, 0, 1)
y_train_encoded = pd.Series(np.where(y_train == 'Decrease', -1, np.where(y_train == 'Hold', 0, 1)))
y_valid_encoded = pd.Series(np.where(y_valid == 'Decrease', -1, np.where(y_valid == 'Hold', 0, 1)))

# Center and scale the independent variables
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

# Build the RNN model with additional modifications to obtain feature importances
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('lstm_units', min_value=32, max_value=128, step=32),
                   input_shape=(sequence_length, 1),
                   return_sequences=True))
    model.add(LSTM(units=hp.Int('lstm_units', min_value=16, max_value=64, step=16)))
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4])),
                  metrics=['accuracy'])

    return model

# Prepare the input sequences for the RNN model (assuming sequence_length is defined)
def prepare_sequences(data, sequence_length):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length + 1):
        sequences.append(data[i:i+sequence_length])
        targets.append(data[i+sequence_length-1])
    return np.array(sequences), np.array(targets)

sequence_length = 10  # Define the length of the input sequence
X_train_seq, y_train_seq = prepare_sequences(y_train_encoded, sequence_length)
X_valid_seq, y_valid_seq = prepare_sequences(y_valid_encoded, sequence_length)

# Build the RNN model with additional modifications to obtain feature importances
model = Sequential()
model.add(LSTM(64, input_shape=(sequence_length, 1), return_sequences=True))
model.add(LSTM(32))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_seq, y_train_seq, epochs=10, batch_size=32, validation_data=(X_valid_seq, y_valid_seq))

# Evaluate the model on the validation set
y_train_pred = model.predict(X_train_seq).argmax(axis=1)
y_valid_pred = model.predict(X_valid_seq).argmax(axis=1)

# Calculate Mean Squared Error (MSE) for training and validation sets
mse_train = mean_squared_error(y_train_seq, y_train_pred)
mse_valid = mean_squared_error(y_valid_seq, y_valid_pred)
print("Training Mean Squared Error (MSE):", mse_train)
print("Validation Mean Squared Error (MSE):", mse_valid)

# Calculate Root Mean Squared Error (RMSE) for training and validation sets
rmse_train = np.sqrt(mse_train)
rmse_valid = np.sqrt(mse_valid)
print("Training Root Mean Squared Error (RMSE):", rmse_train)
print("Validation Root Mean Squared Error (RMSE):", rmse_valid)

# Calculate R-squared (R^2) for training and validation sets
r2_train = r2_score(y_train_seq, y_train_pred)
r2_valid = r2_score(y_valid_seq, y_valid_pred)
print("Training R^2:", r2_train)
print("Validation R^2:", r2_valid)

# Calculate the number of samples and features
n_train, p_train = X_train_seq.shape[0], X_train_seq.shape[1]
n_valid, p_valid = X_valid_seq.shape[0], X_valid_seq.shape[1]

# Calculate adjusted R-squared for training and validation sets
adj_r2_train = 1 - ((1 - r2_train) * (n_train - 1) / (n_train - p_train - 1))
adj_r2_valid = 1 - ((1 - r2_valid) * (n_valid - 1) / (n_valid - p_valid - 1))
print("Training Adjusted R^2:", adj_r2_train)
print("Validation Adjusted R^2:", adj_r2_valid)

# Show training and validation set accuracy
accuracy_train = (y_train_seq == y_train_pred).mean()
accuracy_valid = (y_valid_seq == y_valid_pred).mean()
print("Training Accuracy:", accuracy_train)
print("Validation Accuracy:", accuracy_valid)

Epoch 1/10
6/6 [==============================] - 6s 251ms/step - loss: 1.0475 - accuracy: 0.6503 - val_loss: 0.8374 - val_accuracy: 1.0000
Epoch 2/10
6/6 [==============================] - 0s 30ms/step - loss: 0.7004 - accuracy: 1.0000 - val_loss: 0.4848 - val_accuracy: 1.0000
Epoch 3/10
6/6 [==============================] - 0s 18ms/step - loss: 0.3495 - accuracy: 1.0000 - val_loss: 0.1727 - val_accuracy: 1.0000
Epoch 4/10
6/6 [==============================] - 0s 17ms/step - loss: 0.1089 - accuracy: 1.0000 - val_loss: 0.0463 - val_accuracy: 1.0000
Epoch 5/10
6/6 [==============================] - 0s 17ms/step - loss: 0.0309 - accuracy: 1.0000 - val_loss: 0.0160 - val_accuracy: 1.0000
Epoch 6/10
6/6 [==============================] - 0s 23ms/step - loss: 0.0118 - accuracy: 1.0000 - val_loss: 0.0074 - val_accuracy: 1.0000
Epoch 7/10
6/6 [==============================] - 0s 22ms/step - loss: 0.0060 - accuracy: 1.0000 - val_loss: 0.0043 - val_accuracy: 1.0000
Epoch 8/10
6/6 [==========